In [1]:
import pandas as pd

DATA_PATH = '../data/'
filename  = 'estrepublicain_annee_1999.csv'

df = pd.read_csv(DATA_PATH + filename)
# df = df[0:1000]

# sampler 

df = df.sample(frac= 0.5)
df.reset_index(inplace = True, drop = True)
print(df.shape)

(15120, 1)


In [2]:
df
print("= dimensions")
print(df.shape)
print("= 5 premieres rangés")
print(df.head())
print("= Noms des colonnes")
print(df.columns)


= dimensions
(15120, 1)
= 5 premieres rangés
                                                text
0  Le chemin longe les nombreux bassins d'élevage...
1  Le secours et le salut des âmes pour les habit...
2  L'association « Le Haut Fer » organise deux ma...
3  On les trouve très actifs en ce moment les jeu...
4  NANCY. - La dernière épreuve venait de boucler...
= Noms des colonnes
Index(['text'], dtype='object')


In [4]:
# Changer le nom de la colonne
df.columns = ['paragraphe']
df.columns

Index(['paragraphe'], dtype='object')

In [5]:
# puis renommer colonne en 'text'

df.columns = ['text']
df.columns

Index(['text'], dtype='object')

In [6]:
# Ne prendre que les 1000 premieres rangées

# print("avant: {}".format(df.shape))
# df = df[0:1000]

# print("apres: {}".format(df.shape))



In [7]:
# Tokenization
from nltk import word_tokenize

# La phrase
sentence = "les médiateurs confirment « l'absence de solution parfaite ». "

tokens = word_tokenize(sentence)

print(tokens)
tokens_on_space = sentence.split(' ')
print(tokens_on_space)

['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '»', '.']
['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '».', '']


In [8]:
# ponctuation

import string

print("Liste original de signe de ponctuations")
print("\t{}".format(string.punctuation))

sentence = "les médiateurs confirment « l'absence de solution parfaite ». "
punctuation_chars = string.punctuation + "«»"

#  Construction d'un dict { '~':' ', '$': ' ', ... }
dict_ponctuation = {}
for k in punctuation_chars:
    dict_ponctuation[k] = ' '

print(dict_ponctuation)

# # L'operateur de translation
translator = str.maketrans(dict_ponctuation)

new_sentence = sentence.translate(translator)
print(sentence)
print(new_sentence)


Liste original de signe de ponctuations
	!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{'!': ' ', '"': ' ', '#': ' ', '$': ' ', '%': ' ', '&': ' ', "'": ' ', '(': ' ', ')': ' ', '*': ' ', '+': ' ', ',': ' ', '-': ' ', '.': ' ', '/': ' ', ':': ' ', ';': ' ', '<': ' ', '=': ' ', '>': ' ', '?': ' ', '@': ' ', '[': ' ', '\\': ' ', ']': ' ', '^': ' ', '_': ' ', '`': ' ', '{': ' ', '|': ' ', '}': ' ', '~': ' ', '«': ' ', '»': ' '}
les médiateurs confirment « l'absence de solution parfaite ». 
les médiateurs confirment   l absence de solution parfaite    


In [9]:
# stopwords
tokens = word_tokenize(new_sentence)
print(tokens)

from nltk.corpus import stopwords


# print("=== stopwords - français:")
# print(sorted(stopwords.words('french')))

list_stopwords = stopwords.words('french') + ['les', 'de']

# # equivalent:
# stopwords = stopwords.words('french')
# stopwords.append('les')
# stopwords.append('de')


# 
tokens_sans_stopwords = [w for w in tokens if (w not in list_stopwords) ]

## Equivalent 
tokens_sans_stopwords = []
for w in tokens:
    if w not in list_stopwords:
        tokens_sans_stopwords.append(w)



print(tokens_sans_stopwords)

['les', 'médiateurs', 'confirment', 'l', 'absence', 'de', 'solution', 'parfaite']
['médiateurs', 'confirment', 'absence', 'solution', 'parfaite']


In [10]:
# Enlevons la ponctuation

df['text_no_punctuation'] = df.text.apply(lambda 
    r : ( r.translate(translator) ) 
)
df.head()


,text,text_no_punctuation
0,Le chemin longe les nombreux bassins d'élevage...,Le chemin longe les nombreux bassins d élevage...
1,Le secours et le salut des âmes pour les habit...,Le secours et le salut des âmes pour les habit...
2,L'association « Le Haut Fer » organise deux ma...,L association Le Haut Fer organise deux ma...
3,On les trouve très actifs en ce moment les jeu...,On les trouve très actifs en ce moment les jeu...
4,NANCY. - La dernière épreuve venait de boucler...,NANCY La dernière épreuve venait de boucler...


In [11]:
# Tokenizer

df['tokens_all']  = df.text_no_punctuation.apply(
    lambda r : word_tokenize(r.lower())
)

df.head()

,text,text_no_punctuation,tokens_all
0,Le chemin longe les nombreux bassins d'élevage...,Le chemin longe les nombreux bassins d élevage...,"[le, chemin, longe, les, nombreux, bassins, d,..."
1,Le secours et le salut des âmes pour les habit...,Le secours et le salut des âmes pour les habit...,"[le, secours, et, le, salut, des, âmes, pour, ..."
2,L'association « Le Haut Fer » organise deux ma...,L association Le Haut Fer organise deux ma...,"[l, association, le, haut, fer, organise, deux..."
3,On les trouve très actifs en ce moment les jeu...,On les trouve très actifs en ce moment les jeu...,"[on, les, trouve, très, actifs, en, ce, moment..."
4,NANCY. - La dernière épreuve venait de boucler...,NANCY La dernière épreuve venait de boucler...,"[nancy, la, dernière, épreuve, venait, de, bou..."


In [12]:
# remove stopwords

def remove_stopword(tokens):
     return [w for w in tokens if (w not in list_stopwords) ]

# Verifier que ca marche
remove_stopword(tokens)

# appliquer a la dataframe

df['tokens'] = df.tokens_all.apply(
    lambda tks : remove_stopword(tks) 
)

df.head()

,text,text_no_punctuation,tokens_all,tokens
0,Le chemin longe les nombreux bassins d'élevage...,Le chemin longe les nombreux bassins d élevage...,"[le, chemin, longe, les, nombreux, bassins, d,...","[chemin, longe, nombreux, bassins, élevage, am..."
1,Le secours et le salut des âmes pour les habit...,Le secours et le salut des âmes pour les habit...,"[le, secours, et, le, salut, des, âmes, pour, ...","[secours, salut, âmes, habitants, marquisat, b..."
2,L'association « Le Haut Fer » organise deux ma...,L association Le Haut Fer organise deux ma...,"[l, association, le, haut, fer, organise, deux...","[association, haut, fer, organise, deux, manif..."
3,On les trouve très actifs en ce moment les jeu...,On les trouve très actifs en ce moment les jeu...,"[on, les, trouve, très, actifs, en, ce, moment...","[trouve, très, actifs, moment, jeunes, agricul..."
4,NANCY. - La dernière épreuve venait de boucler...,NANCY La dernière épreuve venait de boucler...,"[nancy, la, dernière, épreuve, venait, de, bou...","[nancy, dernière, épreuve, venait, boucler, st..."


In [13]:
# enlever les colonnes intermediaires

print(df.columns)

df = df[['text', 'tokens']]

print(df.columns)

Index(['text', 'text_no_punctuation', 'tokens_all', 'tokens'], dtype='object')
Index(['text', 'tokens'], dtype='object')


In [14]:
# nombre de tokens par rangée
pd.options.mode.chained_assignment = None
df['token_count'] = df.tokens.apply( lambda r : len(r) )

print(df.head())

# repartition du nombre de tokens
df.token_count.describe()


                                                text  \
0  Le chemin longe les nombreux bassins d'élevage...   
1  Le secours et le salut des âmes pour les habit...   
2  L'association « Le Haut Fer » organise deux ma...   
3  On les trouve très actifs en ce moment les jeu...   
4  NANCY. - La dernière épreuve venait de boucler...   

                                              tokens  token_count  
0  [chemin, longe, nombreux, bassins, élevage, am...          103  
1  [secours, salut, âmes, habitants, marquisat, b...           62  
2  [association, haut, fer, organise, deux, manif...           59  
3  [trouve, très, actifs, moment, jeunes, agricul...           60  
4  [nancy, dernière, épreuve, venait, boucler, st...           60  


count    15120.000000
mean        80.555225
std         35.676722
min         44.000000
25%         64.000000
50%         72.000000
75%         86.000000
max       1011.000000
Name: token_count, dtype: float64

In [15]:
# quel document a 955 tokens?

condition = (df.token_count == 955)
print(df[condition].text.values)


[ "161. Brisset (CSA BA 133) ; 162. Tascone ; 163. Leroy (SNCF) ; 164. Cassin (Tomblaine) ; 165. Aubert (Frouard) ; 166. Pascaot ; 167. Lagrange (Messein) ; 168. Vermandé (Messein) ; 169. Rouyer (Commercy) ; 170. Chrisment (Nancy) ; 171. Lepeltier (Ecrouves) ; 172. Jacquot ; 173. Delery ; 174. Billy ; 175. Lamy ; 176. Charpentier (Ludres) ; 177. Tousch (Gondrecourt) ; 178. Carbillet (La Poste) ; 179. Brunot (Philips) ; 180. Benyoussef ; 181. Starosse (Allamps) ; 182. Seiler (Vandoeuvre) ; 183.Discristofano ; 184. Fossard (Vandoeuvre) ; 185. Boyer ; 186. Leroy ; 187. Carreiras (Golbey) ; 188. Evrard ; 189. Vagne (CSA BA 133) ; 190. Georges (Tomblaine) ; 191. Aubert (Maidières) ; 192. Leroux ; 193. Geoffroy ; 194. Clausse (Chaudenay) ; 195. Deuze (Villers) ; 196. P. Knapek (Ecrouves) ; 197. F. Kanpek (Toul) ; 198. Noël ; 199. Maire (Varangéville) ; 200. Mercky ; 201. Gaumer ; 202. Vincent (Sommerviller) ; 203. Socha (Aingeray) ; 204. Boulot (Blénod) ; 205. Houin (Amicale CHU) ; 206. Borg

In [16]:
# quel index
print(df[condition].index)

# 
condition = (df.token_count < 800)
print(df[condition].shape)


Int64Index([5457], dtype='int64')
(15115, 3)


In [17]:
# enlever le paragraphe le plus long

condition_filtrage = df.token_count < 955
print(df[condition_filtrage].shape)

df = df[condition_filtrage]
print(df[condition_filtrage].shape)

print(df.token_count.describe())


(15118, 3)
(15118, 3)
count    15118.000000
mean        80.435838
std         34.134027
min         44.000000
25%         64.000000
50%         72.000000
75%         86.000000
max        949.000000
Name: token_count, dtype: float64


/home/alexis/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [18]:
# Gensim - Vocabulaire

from gensim import corpora, models
dictionary  = corpora.Dictionary(df.tokens)
print(dictionary)

Dictionary(82416 unique tokens: ['chemin', 'longe', 'nombreux', 'bassins', 'élevage']...)


In [19]:
# corpus_gensim

df['corpus_gensim'] = df.tokens.apply(lambda d : dictionary.doc2bow(d))

print(df[['text','corpus_gensim']] .head().values)


[[ "Le chemin longe les nombreux bassins d'élevage. Les amateurs de poissons d'eau douce pourront y faire un détour et ramener à la maison, moyennant finance, quelques belles truites arc-en-ciel. Le chemin suit le ruisseau jusqu'à un vaste parking équipé pour le pique-nique avec un chalet confortable, demander la clé à la mairie. Depuis le parking, suivre le chemin de débardage qui prend rapidement de l'altitude, passe devant un réservoir et rencontre un chemin de crête. Celui-ci, par la droite, sort du bois, dessert la maison des Poteys et se maintient sur les collines dominant Xertigny. Laisser un premier chemin à gauche puis deux autres à droite pour traverser la route de Plombières. Emprunter la route sur 100 m à droite et s'engager dans le petit chemin de terre qui longe le bois du Fays Richard et conduit à Sainte-Valburge. Regagner le centre du village. L'ancien château de la brasserie qui abrite les services municipaux est entouré d'un arboretum. Vous pourrez y découvrir un spéc

In [20]:
corpus_gensim = [c for c in df.corpus_gensim ]


In [26]:
num_topics= 20

# Le model LDA
lda = models.LdaModel(corpus_gensim,
    id2word      = dictionary,
    num_topics   = num_topics,
    alpha        = 'asymmetric',
    eta          = 'auto',
    passes       = 2,
    iterations   = 20
)

for t in lda.show_topics(num_topics=num_topics, formatted=True, log = False):
    print("\n=== topic #{}".format(t[0]))
    print(t[1].replace('*', ': ').replace(' +',', ').replace('"',''))


=== topic #0
0.022: a,  0.008: plus,  0.007: cette,  0.005: deux,  0.004: être,  0.004: tout,  0.004: faire,  0.004: président,  0.004: fait,  0.003: bien

=== topic #1
0.014: a,  0.011: enfants,  0.008: cette,  0.007: ils,  0.007: plus,  0.006: école,  0.006: élèves,  0.005: année,  0.005: jeunes,  0.005: tous

=== topic #2
0.037: a,  0.008: plus,  0.008: fait,  0.007: tout,  0.007: bien,  0.006: comme,  0.006: ils,  0.006: ans,  0.005: deux,  0.005: très

=== topic #3
0.009: a,  0.004: deux,  0.004: plus,  0.003: mètres,  0.003: circulation,  0.003: véhicule,  0.003: poids,  0.003: parking,  0.003: voiture,  0.002: hauteur

=== topic #4
0.012: a,  0.009: plus,  0.007: eau,  0.005: ils,  0.004: bois,  0.004: tout,  0.003: comme,  0.003: cette,  0.003: après,  0.003: faire

=== topic #5
0.040: f,  0.021: 000,  0.009: conseil,  0.008: a,  0.007: 1,  0.006: 500,  0.006: travaux,  0.005: francs,  0.005: millions,  0.004: 2

=== topic #6
0.038: 6,  0.023: 5,  0.021: 2,  0.020: 4,  0.019: 